In [1]:
import os
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Optional to save generator, checkpoint to your google drive
# Prevents data being erased if colab session disconnects
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Dissertation-project/GM-VQVAE

[Errno 2] No such file or directory: '/content/drive/MyDrive/Dissertation-project/GM-VQVAE'
/content


In [ ]:
#! git clone https://github.com/rosinality/vq-vae-2-pytorch.git

Cloning into 'vq-vae-2-pytorch'...
remote: Enumerating objects: 82, done.
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (82/82), 6.88 MiB | 16.97 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
#! pip install torch

## Loading data:

In [3]:
"""
The following is an import of PyTorch libraries.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import numpy as np
#import nibabel as ni
import os, shutil
import time
import random
import pandas as pd
import numpy as np
import os
import cv2
import numpy as np
import os
import cv2
from scipy import ndimage
import torchvision.transforms.functional as TF
import random
import matplotlib.pyplot as plt

"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Visualization

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def display_image3d(x):
  z1,x1,y1 = x.nonzero()
  fig = plt.figure()
  ax = plt.axes(projection='3d')
  ax.scatter(x1, y1, z1, c=z1, cmap='viridis', alpha=0.7)
  plt.show()

## Data processing

In [5]:
def crop_around_centroid(array, dim1):
  i, j = ndimage.center_of_mass(array)
  i, j = int(i), int(j)
  w = int(dim1/2)
  imin = max(0,i-w)
  imax = min(array.shape[0],i+w+1)
  jmin = max(0,j-w)
  jmax = min(array.shape[1],j+w+1)
  crop =  array[imin:imax,jmin:jmax]
  return crop

def standard_resize2d(image, dim):
  resize_x,  resize_y = dim[0], dim[1]
  img_sm = cv2.resize(image, (resize_x, resize_y), interpolation=cv2.INTER_CUBIC)

  return img_sm


def split_train_test(dir, ratio_test=0.15):
    if not os.path.exists(os.path.join(dir, "train")): os.mkdir(os.path.join(dir, "train"))
    if not os.path.exists(os.path.join(dir, "test")): os.mkdir(os.path.join(dir, "test"))

    images_list = [i for i in os.listdir(dir) if i.endswith(".nii")]

    random.shuffle(images_list)
    threshold = int(len(images_list)*ratio_test)
    train_list = images_list[:-threshold]
    test_list = images_list[-threshold:]

    for i in train_list:
        shutil.move(os.path.join(dir, i), os.path.join(dir, "train", i))
    for i in test_list:
        shutil.move(os.path.join(dir, i), os.path.join(dir, "test", i))

def save_data_to_csv(dir, z):
    pd.DataFrame(z).to_csv(dir, header=None, index=False)

def load_data_images(path, batch_size):
    filenames = [i for i in os.listdir(path) if i.endswith(".npy")] #and i.startswith("norm_023_S_0030")
    random.shuffle(filenames, random.random)
    n = 0
    while n < len(filenames):
        batch_image = []
        for i in range(n, n + batch_size):
            #print(filenames[i])
            if i >= len(filenames):
                ##n = i
                break

            image = np.load(os.path.join(path, filenames[i]), allow_pickle=True)#[1, ...]
            image = np.where(image >= 1e-3, image, 0.0)
            image = crop_around_centroid(image, dim1=240)
            image = np.pad(image, ((1,0), (1,0)), "constant", constant_values=0)
            dim = (256,256)
            image = torch.Tensor(standard_resize2d(image, dim))
            #image = random_rotate_transforms(image)
            image = torch.reshape(image, (1,1, 256, 256))
            image = (image - torch.min(image)) / (torch.max(image) - torch.min(image))
            image = torch.where(image >= 0.1, image, 0.0)
            batch_image.append(image)

        n += batch_size
        batch_image = torch.cat(batch_image, axis=0)
        yield batch_image

## Util Functions

In [6]:
import math
import pickle

import torch
from torch import distributed as dist
from torch.utils import data


LOCAL_PROCESS_GROUP = None


def is_primary():
    return get_rank() == 0


def get_rank():
    if not dist.is_available():
        return 0

    if not dist.is_initialized():
        return 0

    return dist.get_rank()


def get_local_rank():
    if not dist.is_available():
        return 0

    if not dist.is_initialized():
        return 0

    if LOCAL_PROCESS_GROUP is None:
        raise ValueError("tensorfn.distributed.LOCAL_PROCESS_GROUP is None")

    return dist.get_rank(group=LOCAL_PROCESS_GROUP)


def synchronize():
    if not dist.is_available():
        return

    if not dist.is_initialized():
        return

    world_size = dist.get_world_size()

    if world_size == 1:
        return

    dist.barrier()


def get_world_size():
    if not dist.is_available():
        return 1

    if not dist.is_initialized():
        return 1

    return dist.get_world_size()


def all_reduce(tensor, op=dist.ReduceOp.SUM):
    world_size = get_world_size()

    if world_size == 1:
        return tensor

    dist.all_reduce(tensor, op=op)

    return tensor


def all_gather(data):
    world_size = get_world_size()

    if world_size == 1:
        return [data]

    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    local_size = torch.IntTensor([tensor.numel()]).to("cuda")
    size_list = [torch.IntTensor([1]).to("cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.ByteTensor(size=(max_size,)).to("cuda"))

    if local_size != max_size:
        padding = torch.ByteTensor(size=(max_size - local_size,)).to("cuda")
        tensor = torch.cat((tensor, padding), 0)

    dist.all_gather(tensor_list, tensor)

    data_list = []

    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    world_size = get_world_size()

    if world_size < 2:
        return input_dict

    with torch.no_grad():
        keys = []
        values = []

        for k in sorted(input_dict.keys()):
            keys.append(k)
            values.append(input_dict[k])

        values = torch.stack(values, 0)
        dist.reduce(values, dst=0)

        if dist.get_rank() == 0 and average:
            values /= world_size

        reduced_dict = {k: v for k, v in zip(keys, values)}

    return reduced_dict


def data_sampler(dataset, shuffle, distributed):
    if distributed:
        return data.distributed.DistributedSampler(dataset, shuffle=shuffle)

    if shuffle:
        return data.RandomSampler(dataset)

    else:
        return data.SequentialSampler(dataset)

## GMM component

In [7]:
"""
The following is an import of PyTorch libraries.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import numpy as np
#import nibabel as ni
import os, shutil
import time
import random
import pandas as pd
import numpy as np
import os
import cv2
import numpy as np
import os
import cv2
from scipy import ndimage
import torchvision.transforms.functional as TF
import random
import matplotlib.pyplot as plt
import torch
import torch.nn.init as init
from torch import nn
from torch.nn import functional as F
#from networks.Layers import *


"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
from torch import nn
from torch.nn import functional as F

# Flatten layer
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(x.size(0), -1)

# Reshape layer
class Reshape(nn.Module):
  def __init__(self, outer_shape):
    super(Reshape, self).__init__()
    self.outer_shape = outer_shape
  def forward(self, x):
    return x.view(x.size(0), *self.outer_shape)

# Sample from the Gumbel-Softmax distribution and optionally discretize.
class GumbelSoftmax(nn.Module):

  def __init__(self, f_dim, c_dim):
    super(GumbelSoftmax, self).__init__()
    self.logits = nn.Linear(f_dim, c_dim)
    self.f_dim = f_dim
    self.c_dim = c_dim

  def sample_gumbel(self, shape, is_cuda=False, eps=1e-20):
    U = torch.rand(shape)
    if is_cuda:
      U = U.to(device)
    return -torch.log(-torch.log(U + eps) + eps)

  def gumbel_softmax_sample(self, logits, temperature):
    y = logits + self.sample_gumbel(logits.size(), logits.is_cuda)
    return F.softmax(y / temperature, dim=-1)

  def gumbel_softmax(self, logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    #categorical_dim = 10
    y = self.gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard

  def forward(self, x, temperature=1.0, hard=False):
    logits = self.logits(x).view(-1, self.c_dim)
    prob = F.softmax(logits, dim=-1)
    y = self.gumbel_softmax(logits, temperature, hard)
    return logits, prob, y

# Sample from a Gaussian distribution
class Gaussian(nn.Module):
  def __init__(self, in_dim, z_dim):
    super(Gaussian, self).__init__()
    self.mu = nn.Linear(in_dim, z_dim)
    self.var = nn.Linear(in_dim, z_dim)

  def reparameterize(self, mu, var):
    std = torch.sqrt(var + 1e-10)
    noise = torch.randn_like(std)
    z = mu + noise * std
    return z

  def forward(self, x):
    mu = self.mu(x)
    var = F.softplus(self.var(x))
    z = self.reparameterize(mu, var)
    return mu, var, z

# Inference Network
class InferenceNet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(InferenceNet, self).__init__()

    # q(y|x)
    self.inference_qyx = torch.nn.ModuleList([
        nn.Linear(x_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        GumbelSoftmax(512, y_dim)
    ])

    # q(z|y,x)
    self.inference_qzyx = torch.nn.ModuleList([
        nn.Linear(x_dim + y_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        Gaussian(512, z_dim)
    ])

  # q(y|x)
  def qyx(self, x, temperature, hard):
    num_layers = len(self.inference_qyx)
    for i, layer in enumerate(self.inference_qyx):
      if i == num_layers - 1:
        #last layer is gumbel softmax
        x = layer(x, temperature, hard)
      else:
        x = layer(x)
    return x

  # q(z|x,y)
  def qzxy(self, x, y):
    concat = torch.cat((x, y), dim=1)
    for layer in self.inference_qzyx:
      concat = layer(concat)
    return concat

  def forward(self, x, temperature=1.0, hard=0):
    #x = Flatten(x)

    # q(y|x)
    logits, prob, y = self.qyx(x, temperature, hard)

    # q(z|x,y)
    mu, var, z = self.qzxy(x, y)

    output = {'mean': mu, 'var': var, 'gaussian': z,
              'logits': logits, 'prob_cat': prob, 'categorical': y}
    return output


# Generative Network
class GenerativeNet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(GenerativeNet, self).__init__()

    # p(z|y)
    self.y_mu = nn.Linear(y_dim, z_dim)
    self.y_var = nn.Linear(y_dim, z_dim)
    # p(x|z)
    self.generative_pxz = torch.nn.ModuleList([
        nn.Linear(z_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, x_dim),
        torch.nn.Sigmoid()
    ])

  # p(z|y)
  def pzy(self, y):
    y_mu = self.y_mu(y)
    y_var = F.softplus(self.y_var(y))
    return y_mu, y_var

  # p(x|z)
  def pxz(self, z):
    for layer in self.generative_pxz:
      z = layer(z)
    return z

  def forward(self, z, y):
    # p(z|y)
    y_mu, y_var = self.pzy(y)

    # p(x|z)
    x_rec = self.pxz(z)

    output = {'y_mean': y_mu, 'y_var': y_var, 'x_rec': x_rec}
    return output


# GMVAE Network
class GMVAENet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(GMVAENet, self).__init__()
    self.inference = InferenceNet(x_dim, z_dim, y_dim)
    self.generative = GenerativeNet(x_dim, z_dim, y_dim)

    # weight initialization
    for m in self.modules():
      if type(m) == nn.Linear or type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        torch.nn.init.xavier_normal_(m.weight)
        if m.bias.data is not None:
          init.constant_(m.bias, 0)

  def forward(self, x, temperature=1.0, hard=0):
    x = x.view(x.size(0), -1)
    out_inf = self.inference(x, temperature, hard)
    z, y = out_inf['gaussian'], out_inf['categorical']
    #out_gen = self.generative(z, y)

    # merge output
    #output = out_inf
    #for key, value in out_gen.items():
      #output[key] = value

    return z #output

## GM-VQVAE model

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

import distributed as dist_fn


# Copyright 2018 The Sonnet Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================


# Borrowed from https://github.com/deepmind/sonnet and ported it to PyTorch


class Quantize(nn.Module):
    def __init__(self, dim, n_embed, decay=0.99, eps=1e-5):
        super().__init__()

        self.dim = dim
        self.n_embed = n_embed
        self.decay = decay
        self.eps = eps

        embed = torch.randn(dim, n_embed)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(n_embed))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, input):
        flatten = input.reshape(-1, self.dim)
        dist = (
            flatten.pow(2).sum(1, keepdim=True)
            - 2 * flatten @ self.embed
            + self.embed.pow(2).sum(0, keepdim=True)
        )
        _, embed_ind = (-dist).max(1)
        embed_onehot = F.one_hot(embed_ind, self.n_embed).type(flatten.dtype)
        embed_ind = embed_ind.view(*input.shape[:-1])
        quantize = self.embed_code(embed_ind)

        if self.training:
            embed_onehot_sum = embed_onehot.sum(0)
            embed_sum = flatten.transpose(0, 1) @ embed_onehot

            all_reduce(embed_onehot_sum)
            all_reduce(embed_sum)

            self.cluster_size.data.mul_(self.decay).add_(
                embed_onehot_sum, alpha=1 - self.decay
            )
            self.embed_avg.data.mul_(self.decay).add_(embed_sum, alpha=1 - self.decay)
            n = self.cluster_size.sum()
            cluster_size = (
                (self.cluster_size + self.eps) / (n + self.n_embed * self.eps) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            self.embed.data.copy_(embed_normalized)

        diff = (quantize.detach() - input).pow(2).mean()
        quantize = input + (quantize - input).detach()

        return quantize, diff, embed_ind

    def embed_code(self, embed_id):
        return F.embedding(embed_id, self.embed.transpose(0, 1))


class ResBlock(nn.Module):
    def __init__(self, in_channel, channel):
        super().__init__()

        self.conv = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channel, channel, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel, in_channel, 1),
        )

    def forward(self, input):
        out = self.conv(input)
        out += input

        return out


class Encoder(nn.Module):
    def __init__(self, in_channel, channel, n_res_block, n_res_channel, stride):
        super().__init__()

        if stride == 4:
            blocks = [
                nn.Conv2d(in_channel, channel // 2, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 2, channel, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel, channel, 3, padding=1),
            ]

        elif stride == 2:
            blocks = [
                nn.Conv2d(in_channel, channel // 2, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 2, channel, 3, padding=1),
            ]

        for i in range(n_res_block):
            blocks.append(ResBlock(channel, n_res_channel))

        blocks.append(nn.ReLU(inplace=True))

        self.blocks = nn.Sequential(*blocks)

    def forward(self, input):
        return self.blocks(input)


class Decoder(nn.Module):
    def __init__(
        self, in_channel, out_channel, channel, n_res_block, n_res_channel, stride
    ):
        super().__init__()

        blocks = [nn.Conv2d(in_channel, channel, 3, padding=1)]

        for i in range(n_res_block):
            blocks.append(ResBlock(channel, n_res_channel))

        blocks.append(nn.ReLU(inplace=True))

        if stride == 4:
            blocks.extend(
                [
                    nn.ConvTranspose2d(channel, channel // 2, 4, stride=2, padding=1),
                    nn.ReLU(inplace=True),
                    nn.ConvTranspose2d(
                        channel // 2, out_channel, 4, stride=2, padding=1
                    ),
                ]
            )

        elif stride == 2:
            blocks.append(
                nn.ConvTranspose2d(channel, out_channel, 4, stride=2, padding=1)
            )

        self.blocks = nn.Sequential(*blocks)

    def forward(self, input):
        return self.blocks(input)


class VQVAE(nn.Module):
    def __init__(
        self,
        in_channel=1,
        channel=128,
        n_res_block=2,
        n_res_channel=32,
        embed_dim=64,
        n_embed=512,
        decay=0.99,
    ):
        super().__init__()
        self.enc_b = Encoder(in_channel, channel, n_res_block, n_res_channel, stride=4)
        self.enc_t = Encoder(channel, channel, n_res_block, n_res_channel, stride=2)
        self.quantize_conv_t = nn.Conv2d(channel, embed_dim, 1)
        self.quantize_t = Quantize(embed_dim, n_embed)
        self.dec_t = Decoder(
            embed_dim, embed_dim, channel, n_res_block, n_res_channel, stride=2
        )
        self.quantize_conv_b = nn.Conv2d(embed_dim + channel, embed_dim, 1)
        self.quantize_b = Quantize(embed_dim, n_embed)
        self.upsample_t = nn.ConvTranspose2d(
            embed_dim, embed_dim, 4, stride=2, padding=1
        )
        self.dec = Decoder(
            embed_dim + embed_dim,
            in_channel,
            channel,
            n_res_block,
            n_res_channel,
            stride=4,
        )


    def forward(self, input):

        quant_t, quant_b, diff, _, _ = self.encode(input)
        #GMM  representation
        quant_t_scale, quant_b_scale = self.gmm_represent(quant_t, gaussian_size=quant_t.shape[-1]), self.gmm_represent(quant_b, gaussian_size=quant_b.shape[-1])
        #quant_t_scale, quant_b_scale = torch.mean(torch.flatten(quant_t_scale.permute(0, 2, 1, 3), start_dim=2), dim=2), torch.mean(torch.flatten(quant_b_scale.permute(0, 2, 1, 3), start_dim=2), dim=2)
        dec = self.decode(quant_t * quant_t_scale, quant_b * quant_b_scale)

        return dec, diff

    def encode(self, input):
        enc_b = self.enc_b(input)
        enc_t = self.enc_t(enc_b)

        quant_t = self.quantize_conv_t(enc_t).permute(0, 2, 3, 1)
        quant_t, diff_t, id_t = self.quantize_t(quant_t)
        quant_t = quant_t.permute(0, 3, 1, 2)
        diff_t = diff_t.unsqueeze(0)

        dec_t = self.dec_t(quant_t)
        enc_b = torch.cat([dec_t, enc_b], 1)

        quant_b = self.quantize_conv_b(enc_b).permute(0, 2, 3, 1)
        quant_b, diff_b, id_b = self.quantize_b(quant_b)
        quant_b = quant_b.permute(0, 3, 1, 2)
        diff_b = diff_b.unsqueeze(0)

        return quant_t, quant_b, diff_t + diff_b, id_t, id_b

    def decode(self, quant_t, quant_b):
        upsample_t = self.upsample_t(quant_t)
        quant = torch.cat([upsample_t, quant_b], 1)
        dec = self.dec(quant)

        return dec

    def decode_code(self, code_t, code_b):
        quant_t = self.quantize_t.embed_code(code_t)
        quant_t = quant_t.permute(0, 3, 1, 2)
        quant_b = self.quantize_b.embed_code(code_b)
        quant_b = quant_b.permute(0, 3, 1, 2)

        dec = self.decode(quant_t, quant_b)

        return dec

    def gmm_represent(self, quant, gaussian_size = 64):
      # GMM for quant_t
      input_size = quant.shape[1] * quant.shape[2] * quant.shape[3]
      num_classes = 1
      gmvae = GMVAENet(input_size, gaussian_size, num_classes).to(device)
      quant_out = quant.view(quant.shape[0], 1, quant.shape[1], -1)
      quant_out = gmvae(quant_out)
      #quant_out = 2 * ((quant_out - torch.min(quant_out))/(torch.max(quant_out)-torch.min(quant_out))) - 1
      return torch.sigmoid(quant_out)

## Loss function

In [ ]:
class KLDivergence(nn.Module):
    "KL divergence between the estimated normal distribution and a prior distribution"
    def __init__(self):
        super(KLDivergence, self).__init__()
        """
        N :  the index N spans all dimensions of input
        N = H x W x D
        """
        self.N = 80*80
    def forward(self, z_mean, z_sigma):
        z_var = z_sigma * 2
        #return (1/self.N) * ( (z_mean**2 + z_var**2 - z_log_var**2 - 1).sum() )
        return 0.5 * ((z_mean**2 + z_var.exp() - z_var - 1).sum())

class L1Loss(nn.Module):

    "Measuring the `Euclidian distance` between prediction and ground truh using `L1 Norm`"
    def __init__(self):
        super(L1Loss, self).__init__()

    def forward(self, x, y):
        N = y.shape[0]*y.shape[1]*y.shape[2]*y.shape[3]*y.shape[4]
        return  ( (x - y).abs()).sum() / N

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

# Training Scripts

## Joint training

In [ ]:
"""
Initialize Hyperparameters
"""
#import nibabel as ni
import matplotlib.pyplot as plt
import numpy as np
import os, glob
import torch
import csv
from tqdm import tqdm
##---------Settings--------------------------
batch_size = 10
lrate = 0.001
epochs = 100
weight_decay = 1e-6 #5e-7

##############
train_path_data = "/content/drive/MyDrive/Dissertation-project/Data/2D-featuremap/all/train/sample"
val_path_data ="/content/drive/MyDrive/Dissertation-project/Data/2D-featuremap/all/val/"
path2save = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n32/joint/model_vae_epoch_{}.pt"
dir_info = '/content/drive/MyDrive/Dissertation-project/infor'
#f = open(os.path.join(dir_info,'model_vae_t1_rectum_2d.csv'),'w',newline='')

####################
verbose = True
log = print if verbose else lambda *x, **i: None
np.random.seed(10)
torch.manual_seed(10)
###################
criterion = nn.MSELoss()


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(" GPU is activated" if device else " CPU is activated")
train_no_images = len(glob.glob(train_path_data + "/*.npy"))
val_no_images = len(glob.glob(val_path_data + "/*.npy"))
print("Number of train fm images: ", train_no_images)
print("Number of val fm images: ", val_no_images)

if __name__=="__main__":
    vae_model = VQVAE()
    vae_model.to(device)
    optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)
    vae_model.apply(weights_init)
    min_valid_loss = np.inf
    train_batch_avg_total_loss_list = []
    val_batch_avg_total_loss_list = []
    beta = 0.25

    for epoch in tqdm(range(epochs)):
        # training phrase
        train_loss_rec_batch, train_loss_KL_batch, train_total_loss_batch = 0, 0, 0
        train_loss_rec_epoch, train_loss_KL_epoch, train_total_loss_epoch = 0, 0, 0

        vae_model.train()
        for train_batch_images in tqdm(load_data_images(train_path_data, batch_size)):
            optimizer.zero_grad()
            #train_batch_images = torch.concat([train_batch_images, train_batch_images, train_batch_images], dim=1)
            train_batch_images = train_batch_images.to(device)
            train_out, train_latent_loss = vae_model(train_batch_images)

            train_recon_loss = criterion(train_out, train_batch_images)
            train_latent_loss = train_latent_loss.mean()

            train_total_loss_batch =  train_recon_loss + beta * train_latent_loss

            # Optimize
            train_total_loss_batch.backward()
            optimizer.step()

            train_total_loss_epoch += train_total_loss_batch.item() * train_batch_images.shape[0]

        train_log_info = (epoch + 1, epochs, train_loss_rec_epoch/train_no_images, train_loss_KL_epoch/train_no_images, train_total_loss_epoch/train_no_images)
        log('%d/%d  Train: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% train_log_info)
        train_batch_avg_total_loss_list.append(train_total_loss_epoch/train_no_images)


        # validation phrase
        val_loss_rec_batch, val_loss_KL_batch, val_total_loss_batch = 0, 0, 0
        val_loss_rec_epoch, val_loss_KL_epoch, val_total_loss_epoch = 0, 0, 0


        vae_model.eval()
        for val_batch_images in tqdm(load_data_images(val_path_data, batch_size)):
            optimizer.zero_grad()
            #val_batch_images = torch.concat([val_batch_images, val_batch_images, val_batch_images], dim=1)
            val_batch_images = val_batch_images.to(device)

            val_batch_images = val_batch_images.to(device)
            val_out, val_latent_loss = vae_model(val_batch_images)
            val_recon_loss = criterion(val_out, val_batch_images)
            val_latent_loss = val_latent_loss.mean()

            val_total_loss_batch =  val_recon_loss + beta * val_latent_loss

            # Optimize
            val_total_loss_batch.backward()
            optimizer.step()
            val_total_loss_epoch += val_total_loss_batch.item() * val_batch_images.shape[0]


        val_log_info = (epoch + 1, epochs,  val_loss_rec_epoch/val_no_images, val_loss_KL_epoch/val_no_images, val_total_loss_epoch/val_no_images)
        log('%d/%d  Validation: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% val_log_info)
        val_batch_avg_total_loss_list.append(val_total_loss_epoch/val_no_images)
        epoch_log_df = pd.DataFrame({"train_loss": train_batch_avg_total_loss_list, "val_loss": val_batch_avg_total_loss_list})
        print(epoch_log_df)
        epoch_log_df.to_csv(os.path.join(dir_info,'model_vae_t1_prostate_2d_16.csv'))
        if min_valid_loss > val_total_loss_epoch/val_no_images:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_total_loss_epoch/val_no_images:.6f}) \t Saving The Model')
          min_valid_loss = val_total_loss_epoch/val_no_images
          # Saving State Dict

        torch.save(vae_model, path2save.format(epoch+1))
        # write csv
        #writer = csv.writer(f)
        #writer.writerow([epoch + 1, '{:04f}'.format(val_total_loss_epoch/train_no_images), '{:04f}'.format(val_total_loss_epoch/val_no_images)])
    #f.close()

<ipython-input-9-6c108b0efd2c>:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


 GPU is activated
Number of train fm images:  5000
Number of val fm images:  775


  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]<ipython-input-5-a8050d94af31>:40: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(filenames, random.random)
0it [00:11, ?it/s]
  0%|          | 0/100 [00:11<?, ?it/s]


RuntimeError: ignored

## Prostate training

In [ ]:
"""
Initialize Hyperparameters
"""
#import nibabel as ni
import matplotlib.pyplot as plt
import numpy as np
import os, glob
import torch
import csv
from tqdm import tqdm
##---------Settings--------------------------
batch_size = 312
lrate = 0.001
epochs = 100
weight_decay = 1e-6 #5e-7

##############
train_path_data = "/content/drive/MyDrive/VAE_GMM/2D-featuremap/Prostate/Prostate_train/sample"
val_path_data ="/content/drive/MyDrive/VAE_GMM/2D-featuremap/Prostate/Prostate_val"
path2save = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae2_checkpoint/prostate/model_vae_epoch_{}.pt"
dir_info = '/content/drive/MyDrive/VAE_GMM/infor'
#f = open(os.path.join(dir_info,'model_vae_t1_rectum_2d.csv'),'w',newline='')

####################
verbose = True
log = print if verbose else lambda *x, **i: None
np.random.seed(10)
torch.manual_seed(10)
###################
criterion = nn.MSELoss()


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(" GPU is activated" if device else " CPU is activated")
train_no_images = len(glob.glob(train_path_data + "/*.npy"))
val_no_images = len(glob.glob(val_path_data + "/*.npy"))
print("Number of train fm images: ", train_no_images)
print("Number of val fm images: ", val_no_images)

if __name__=="__main__":
    vae_model = VQVAE()
    #ckp_path = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae_560.pt"
    #checkpoint =  torch.load(ckp_path)
    #vae_model.load_state_dict(checkpoint)
    vae_model.to(device)
    optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)

    vae_model.apply(weights_init)
    #log(vae_model)
    #optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)
    min_valid_loss = np.inf
    #model, optimizer, start_epoch = load_ckp(ckp_path, vae_model, optimizer)
    train_batch_avg_total_loss_list = []
    val_batch_avg_total_loss_list = []
    beta = 0.25

    for epoch in tqdm(range(epochs)):
        # training phrase
        train_loss_rec_batch, train_loss_KL_batch, train_total_loss_batch = 0, 0, 0
        train_loss_rec_epoch, train_loss_KL_epoch, train_total_loss_epoch = 0, 0, 0

        vae_model.train()
        for train_batch_images in tqdm(load_data_images(train_path_data, batch_size)):
            optimizer.zero_grad()
            #train_batch_images = torch.concat([train_batch_images, train_batch_images, train_batch_images], dim=1)
            train_batch_images = train_batch_images.to(device)
            '''
            plt.imshow(train_batch_images[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_out, train_latent_loss = vae_model(train_batch_images)
            '''
            plt.imshow(train_out[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_recon_loss = criterion(train_out, train_batch_images)
            train_latent_loss = train_latent_loss.mean()

            train_total_loss_batch =  train_recon_loss + beta * train_latent_loss

            # Optimize
            train_total_loss_batch.backward()
            optimizer.step()

            train_total_loss_epoch += train_total_loss_batch.item() * train_batch_images.shape[0]

        train_log_info = (epoch + 1, epochs, train_loss_rec_epoch/train_no_images, train_loss_KL_epoch/train_no_images, train_total_loss_epoch/train_no_images)
        log('%d/%d  Train: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% train_log_info)
        train_batch_avg_total_loss_list.append(train_total_loss_epoch/train_no_images)


        # validation phrase
        val_loss_rec_batch, val_loss_KL_batch, val_total_loss_batch = 0, 0, 0
        val_loss_rec_epoch, val_loss_KL_epoch, val_total_loss_epoch = 0, 0, 0


        vae_model.eval()
        for val_batch_images in tqdm(load_data_images(val_path_data, batch_size)):
            optimizer.zero_grad()
            #val_batch_images = torch.concat([val_batch_images, val_batch_images, val_batch_images], dim=1)
            val_batch_images = val_batch_images.to(device)

            val_batch_images = val_batch_images.to(device)
            val_out, val_latent_loss = vae_model(val_batch_images)
            val_recon_loss = criterion(val_out, val_batch_images)
            val_latent_loss = val_latent_loss.mean()

            val_total_loss_batch =  val_recon_loss + beta * val_latent_loss

            # Optimize
            val_total_loss_batch.backward()
            optimizer.step()
            val_total_loss_epoch += val_total_loss_batch.item() * val_batch_images.shape[0]


        val_log_info = (epoch + 1, epochs,  val_loss_rec_epoch/val_no_images, val_loss_KL_epoch/val_no_images, val_total_loss_epoch/val_no_images)
        log('%d/%d  Validation: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% val_log_info)
        val_batch_avg_total_loss_list.append(val_total_loss_epoch/val_no_images)
        epoch_log_df = pd.DataFrame({"train_loss": train_batch_avg_total_loss_list, "val_loss": val_batch_avg_total_loss_list})
        print(epoch_log_df)
        epoch_log_df.to_csv(os.path.join(dir_info,'model_vae_t1_prostate_2d_16.csv'))
        if min_valid_loss > val_total_loss_epoch/val_no_images:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_total_loss_epoch/val_no_images:.6f}) \t Saving The Model')
          min_valid_loss = val_total_loss_epoch/val_no_images
          # Saving State Dict

        torch.save(vae_model, path2save.format(epoch+1))
        # write csv
        #writer = csv.writer(f)
        #writer.writerow([epoch + 1, '{:04f}'.format(val_total_loss_epoch/train_no_images), '{:04f}'.format(val_total_loss_epoch/val_no_images)])
    #f.close()

<ipython-input-9-1eb2d7fd442d>:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


 GPU is activated
Number of train fm images:  5000
Number of val fm images:  916


  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]<ipython-input-6-b70cece9af57>:40: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(filenames, random.random)


Encoding bottom
torch.Size([312, 128, 64, 64])
Encoding top
torch.Size([312, 128, 32, 32])


0it [03:36, ?it/s]
  0%|          | 0/100 [03:36<?, ?it/s]


OutOfMemoryError: ignored

## Rectum training

In [ ]:
"""
Initialize Hyperparameters
"""
#import nibabel as ni
import matplotlib.pyplot as plt
import numpy as np
import os, glob
import torch
import csv
from tqdm import tqdm
##---------Settings--------------------------
batch_size = 10
lrate = 0.001
epochs = 100
weight_decay = 1e-6 #5e-7

##############
train_path_data = "/content/drive/MyDrive/VAE_GMM/2D-featuremap/Rectum/Rectum_train/sample"
val_path_data ="/content/drive/MyDrive/VAE_GMM/2D-featuremap/Rectum/Rectum_val"
path2save = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae2_checkpoint/prostate/model_vae_epoch_{}.pt"
dir_info = '/content/drive/MyDrive/VAE_GMM/infor'
#f = open(os.path.join(dir_info,'model_vae_t1_rectum_2d.csv'),'w',newline='')

####################
verbose = True
log = print if verbose else lambda *x, **i: None
np.random.seed(10)
torch.manual_seed(10)
###################
criterion = nn.MSELoss()


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(" GPU is activated" if device else " CPU is activated")
train_no_images = len(glob.glob(train_path_data + "/*.npy"))
val_no_images = len(glob.glob(val_path_data + "/*.npy"))
print("Number of train fm images: ", train_no_images)
print("Number of val fm images: ", val_no_images)

if __name__=="__main__":
    vae_model = VQVAE()
    #ckp_path = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae_560.pt"
    #checkpoint =  torch.load(ckp_path)
    #vae_model.load_state_dict(checkpoint)
    vae_model.to(device)
    optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)

    vae_model.apply(weights_init)
    #log(vae_model)
    #optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)
    min_valid_loss = np.inf
    #model, optimizer, start_epoch = load_ckp(ckp_path, vae_model, optimizer)
    train_batch_avg_total_loss_list = []
    val_batch_avg_total_loss_list = []
    beta = 0.25

    for epoch in tqdm(range(epochs)):
        # training phrase
        train_loss_rec_batch, train_loss_KL_batch, train_total_loss_batch = 0, 0, 0
        train_loss_rec_epoch, train_loss_KL_epoch, train_total_loss_epoch = 0, 0, 0

        vae_model.train()
        for train_batch_images in tqdm(load_data_images(train_path_data, batch_size)):
            optimizer.zero_grad()
            #train_batch_images = torch.concat([train_batch_images, train_batch_images, train_batch_images], dim=1)
            train_batch_images = train_batch_images.to(device)
            '''
            plt.imshow(train_batch_images[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_out, train_latent_loss = vae_model(train_batch_images)
            '''
            plt.imshow(train_out[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_recon_loss = criterion(train_out, train_batch_images)
            train_latent_loss = train_latent_loss.mean()

            train_total_loss_batch =  train_recon_loss + beta * train_latent_loss

            # Optimize
            train_total_loss_batch.backward()
            optimizer.step()

            train_total_loss_epoch += train_total_loss_batch.item() * train_batch_images.shape[0]

        train_log_info = (epoch + 1, epochs, train_loss_rec_epoch/train_no_images, train_loss_KL_epoch/train_no_images, train_total_loss_epoch/train_no_images)
        log('%d/%d  Train: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% train_log_info)
        train_batch_avg_total_loss_list.append(train_total_loss_epoch/train_no_images)


        # validation phrase
        val_loss_rec_batch, val_loss_KL_batch, val_total_loss_batch = 0, 0, 0
        val_loss_rec_epoch, val_loss_KL_epoch, val_total_loss_epoch = 0, 0, 0


        vae_model.eval()
        for val_batch_images in tqdm(load_data_images(val_path_data, batch_size)):
            optimizer.zero_grad()
            #val_batch_images = torch.concat([val_batch_images, val_batch_images, val_batch_images], dim=1)
            val_batch_images = val_batch_images.to(device)

            val_batch_images = val_batch_images.to(device)
            val_out, val_latent_loss = vae_model(val_batch_images)
            val_recon_loss = criterion(val_out, val_batch_images)
            val_latent_loss = val_latent_loss.mean()

            val_total_loss_batch =  val_recon_loss + beta * val_latent_loss

            # Optimize
            val_total_loss_batch.backward()
            optimizer.step()
            val_total_loss_epoch += val_total_loss_batch.item() * val_batch_images.shape[0]


        val_log_info = (epoch + 1, epochs,  val_loss_rec_epoch/val_no_images, val_loss_KL_epoch/val_no_images, val_total_loss_epoch/val_no_images)
        log('%d/%d  Validation: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% val_log_info)
        val_batch_avg_total_loss_list.append(val_total_loss_epoch/val_no_images)
        epoch_log_df = pd.DataFrame({"train_loss": train_batch_avg_total_loss_list, "val_loss": val_batch_avg_total_loss_list})
        print(epoch_log_df)
        epoch_log_df.to_csv(os.path.join(dir_info,'model_vae_t1_prostate_2d_16.csv'))
        if min_valid_loss > val_total_loss_epoch/val_no_images:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_total_loss_epoch/val_no_images:.6f}) \t Saving The Model')
          min_valid_loss = val_total_loss_epoch/val_no_images
          # Saving State Dict

        torch.save(vae_model, path2save.format(epoch+1))
        # write csv
        #writer = csv.writer(f)
        #writer.writerow([epoch + 1, '{:04f}'.format(val_total_loss_epoch/train_no_images), '{:04f}'.format(val_total_loss_epoch/val_no_images)])
    #f.close()

 GPU is activated


<ipython-input-9-1eb2d7fd442d>:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


Number of train fm images:  5000
Number of val fm images:  2127


  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]<ipython-input-6-b70cece9af57>:40: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(filenames, random.random)

1it [00:09,  9.33s/it]

Encoding bottom
torch.Size([10, 128, 64, 64])
Encoding top
torch.Size([10, 128, 32, 32])



2it [00:17,  8.50s/it]

Encoding bottom
torch.Size([10, 128, 64, 64])
Encoding top
torch.Size([10, 128, 32, 32])


2it [00:20, 10.15s/it]
  0%|          | 0/100 [00:20<?, ?it/s]


KeyboardInterrupt: ignored

## Bladder training

In [ ]:
"""
Initialize Hyperparameters
"""
#import nibabel as ni
import matplotlib.pyplot as plt
import numpy as np
import os, glob
import torch
import csv
from tqdm import tqdm
##---------Settings--------------------------
batch_size = 10
lrate = 0.001
epochs = 100
weight_decay = 1e-6 #5e-7

##############
train_path_data = "/content/drive/MyDrive/VAE_GMM/2D-featuremap/Bladder/Bladder_train/sample"
val_path_data ="/content/drive/MyDrive/VAE_GMM/2D-featuremap/Bladder/Bladder_val"
path2save = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae2_checkpoint/bladder/model_vae_epoch_{}.pt"
dir_info = '/content/drive/MyDrive/VAE_GMM/infor'
#f = open(os.path.join(dir_info,'model_vae_t1_rectum_2d.csv'),'w',newline='')

####################
verbose = True
log = print if verbose else lambda *x, **i: None
np.random.seed(10)
torch.manual_seed(10)
###################
criterion = nn.MSELoss()


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(" GPU is activated" if device else " CPU is activated")
train_no_images = len(glob.glob(train_path_data + "/*.npy"))
val_no_images = len(glob.glob(val_path_data + "/*.npy"))
print("Number of train fm images: ", train_no_images)
print("Number of val fm images: ", val_no_images)

if __name__=="__main__":
    vae_model = VQVAE()
    #ckp_path = "/content/drive/MyDrive/VAE_GMM/VQ-VAE2/vqvae2/vqvae_560.pt"
    #checkpoint =  torch.load(ckp_path)
    #vae_model.load_state_dict(checkpoint)
    vae_model.to(device)
    optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)

    vae_model.apply(weights_init)
    #log(vae_model)
    #optimizer = torch.optim.Adam(vae_model.parameters(), lr=lrate, weight_decay=weight_decay)
    min_valid_loss = np.inf
    #model, optimizer, start_epoch = load_ckp(ckp_path, vae_model, optimizer)
    train_batch_avg_total_loss_list = []
    val_batch_avg_total_loss_list = []
    beta = 0.25

    for epoch in tqdm(range(epochs)):
        # training phrase
        train_loss_rec_batch, train_loss_KL_batch, train_total_loss_batch = 0, 0, 0
        train_loss_rec_epoch, train_loss_KL_epoch, train_total_loss_epoch = 0, 0, 0

        vae_model.train()
        for train_batch_images in tqdm(load_data_images(train_path_data, batch_size)):
            optimizer.zero_grad()
            #train_batch_images = torch.concat([train_batch_images, train_batch_images, train_batch_images], dim=1)
            train_batch_images = train_batch_images.to(device)
            '''
            plt.imshow(train_batch_images[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_out, train_latent_loss = vae_model(train_batch_images)
            '''
            plt.imshow(train_out[0, 0, ...].cpu().detach().numpy())
            plt.show()
            '''
            train_recon_loss = criterion(train_out, train_batch_images)
            train_latent_loss = train_latent_loss.mean()

            train_total_loss_batch =  train_recon_loss + beta * train_latent_loss

            # Optimize
            train_total_loss_batch.backward()
            optimizer.step()

            train_total_loss_epoch += train_total_loss_batch.item() * train_batch_images.shape[0]

        train_log_info = (epoch + 1, epochs, train_loss_rec_epoch/train_no_images, train_loss_KL_epoch/train_no_images, train_total_loss_epoch/train_no_images)
        log('%d/%d  Train: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% train_log_info)
        train_batch_avg_total_loss_list.append(train_total_loss_epoch/train_no_images)


        # validation phrase
        val_loss_rec_batch, val_loss_KL_batch, val_total_loss_batch = 0, 0, 0
        val_loss_rec_epoch, val_loss_KL_epoch, val_total_loss_epoch = 0, 0, 0


        vae_model.eval()
        for val_batch_images in tqdm(load_data_images(val_path_data, batch_size)):
            optimizer.zero_grad()
            #val_batch_images = torch.concat([val_batch_images, val_batch_images, val_batch_images], dim=1)
            val_batch_images = val_batch_images.to(device)

            val_batch_images = val_batch_images.to(device)
            val_out, val_latent_loss = vae_model(val_batch_images)
            val_recon_loss = criterion(val_out, val_batch_images)
            val_latent_loss = val_latent_loss.mean()

            val_total_loss_batch =  val_recon_loss + beta * val_latent_loss

            # Optimize
            val_total_loss_batch.backward()
            optimizer.step()
            val_total_loss_epoch += val_total_loss_batch.item() * val_batch_images.shape[0]


        val_log_info = (epoch + 1, epochs,  val_loss_rec_epoch/val_no_images, val_loss_KL_epoch/val_no_images, val_total_loss_epoch/val_no_images)
        log('%d/%d  Validation: Reconstruction Loss %.3f| KL Loss %.3f | Total Loss %.3f'% val_log_info)
        val_batch_avg_total_loss_list.append(val_total_loss_epoch/val_no_images)
        epoch_log_df = pd.DataFrame({"train_loss": train_batch_avg_total_loss_list, "val_loss": val_batch_avg_total_loss_list})
        print(epoch_log_df)
        epoch_log_df.to_csv(os.path.join(dir_info,'model_vae_t1_prostate_2d_16.csv'))
        if min_valid_loss > val_total_loss_epoch/val_no_images:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_total_loss_epoch/val_no_images:.6f}) \t Saving The Model')
          min_valid_loss = val_total_loss_epoch/val_no_images
          # Saving State Dict

        torch.save(vae_model, path2save.format(epoch+1))
        # write csv
        #writer = csv.writer(f)
        #writer.writerow([epoch + 1, '{:04f}'.format(val_total_loss_epoch/train_no_images), '{:04f}'.format(val_total_loss_epoch/val_no_images)])
    #f.close()

 GPU is activated


<ipython-input-9-1eb2d7fd442d>:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


Number of train fm images:  5000
Number of val fm images:  664


  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]<ipython-input-6-b70cece9af57>:40: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(filenames, random.random)

1it [00:07,  7.12s/it]

Encoding bottom
torch.Size([10, 128, 64, 64])
Encoding top
torch.Size([10, 128, 32, 32])



2it [00:14,  7.18s/it]

Encoding bottom
torch.Size([10, 128, 64, 64])
Encoding top
torch.Size([10, 128, 32, 32])


2it [00:20, 10.09s/it]
  0%|          | 0/100 [00:20<?, ?it/s]


KeyboardInterrupt: ignored